## Read data

In [1]:
import pandas as pd
od_pairs = pd.read_csv("../data/processed/sample_and_results.csv")
od_pairs.head()

,OID_,SAMPLE_ID,DIST_M,NEAR_RANK,AHA_ID,ZIP,AHA_ID_lon,AHA_ID_lat,ZIP_lon,ZIP_lat,BIN,STATE_ABBR,Google Maps,Bing Maps,ESRI Routing Service,OSRM,OSMNx,ArcGIS Pro,Total_Kilometers_Esri_Desktop
0,2001,1,173.323707,1,6840875.0,81648,-108.806591,40.080259,-108.805998,40.078766,0-15,CO,0.883333,1.466667,4.205097,0.913333,0.733333,0.602626,0.219917
1,2324,324,580.822403,1,6232575.0,15243,-80.066209,40.377315,-80.072264,40.374883,0-15,PA,0.883333,0.866667,0.787713,0.825000,0.716667,0.797469,0.594560
2,2306,306,264.902804,1,6230760.0,16507,-80.087927,42.133525,-80.086650,42.131338,0-15,PA,1.016667,0.616667,0.723206,0.693333,0.350000,0.659159,0.269491
3,3199,1199,160.155908,1,6640004.0,58702,-101.293818,48.232582,-101.295411,48.231612,0-15,ND,1.133333,1.033333,0.650208,0.381667,0.150000,0.617632,0.298608
4,3201,1201,497.392691,1,6640020.0,58502,-100.782259,46.808107,-100.788736,46.808599,0-15,ND,1.450000,1.400000,1.159282,0.875000,0.500000,1.108446,0.494467


## Routing calculation

### Google Maps API

In [3]:
# pip install googlemaps


In [ ]:
%%time
import googlemaps
from tqdm.notebook import tqdm
import time
tqdm.pandas()

xiaokang_key = ""
gmaps = googlemaps.Client(key=xiaokang_key)

def get_drive_time(row):
  # (lat,lon)
  origins = (row["ZIP_Y"],row["ZIP_X"])
  destinations = (row["AHA_ID_Y"],row["AHA_ID_X"])
  mode = "driving"
  res = gmaps.distance_matrix(origins,destinations,mode)
  # print(res)
  time.sleep(1)
  if res['rows'][0]["elements"][0]["status"] == "OK":

    return res['rows'][0]["elements"][0]['duration']['value']
  else:
    print(res['rows'])
    return None

od_pairs.loc[:,"google_drive_time_seconds"] = od_pairs.progress_apply(get_drive_time,axis=1)

### Bing Maps API

In [ ]:
%%time
from tqdm.notebook import tqdm
tqdm.pandas()

BingMapsKey_xiaokang = ""
def get_drive_time(row):
  # (lat,lon)
  mode = "driving"
  # lat,lon;
  url = "https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins=%f,%f&destinations=%f,%f&travelMode=%s&key=%s"
  resp = requests.get(url%(row["ZIP_Y"],row["ZIP_X"],row["AHA_ID_Y"],row["AHA_ID_X"],mode,BingMapsKey_xiaokang))
  res = json.loads(resp.content)

  # res = gmaps.distance_matrix(origins,destinations,mode)
  # print(res)
  time.sleep(1)
  if res['statusDescription'] == "OK":

    return res['resourceSets'][0]['resources'][0]['results'][0]['travelDuration']
  else:
    print(res)
    return None

od_pairs.loc[:,"BingMap_drive_time_seconds"] = od_pairs.progress_apply(get_drive_time,axis=1)

### ESRI Routing Service

In [ ]:
%%time
from tqdm.notebook import tqdm
tqdm.pandas()
esri_key_xiaokang = ""

def get_drive_time(row):
  mode = "driving"
  # lon,lat;
  url = "https://route-api.arcgis.com/arcgis/rest/services/World/Route/NAServer/Route_World/solve?travel_mode=%s&f=json&token=%s&stops=%f,%f;%f,%f"
  resp = requests.get(url%(mode,esri_key_xiaokang,row["ZIP_X"],row["ZIP_Y"],row["AHA_ID_X"],row["AHA_ID_Y"]))
  res = json.loads(resp.content)

  # res = gmaps.distance_matrix(origins,destinations,mode)
  # print(res)

  try:
    return res['directions'][0]['summary']['totalDriveTime']
  except:
    print(res)
    return None
od_pairs.loc[:,"ESRI_drive_time_seconds"] = od_pairs.progress_apply(get_drive_time,axis=1)

### OSRM

In [ ]:
%%time
import time
from tqdm.notebook import tqdm
tqdm.pandas()
def get_drive_time(row):
# lon lat
  mode = "driving"
  url = "http://router.project-osrm.org/route/v1/%s/%f,%f;%f,%f?overview=false"
  resp = requests.get(url%(mode,row["ZIP_X"],row["ZIP_Y"],row["AHA_X"],row["AHA_Y"]))
  res = json.loads(resp.content)

  # res = gmaps.distance_matrix(origins,destinations,mode)
  # print(res)
  time.sleep(1)
  if res['code'] == "Ok":

    return res["routes"][0]['duration']
  else:
    print(res)
    return None

od_pairs.loc[:,"OSRM_drive_time_seconds"] = ak_data.progress_apply(get_drive_time,axis=1)

### OSMnx

In [ ]:
%%time 
run ../src/models/cal_osmnx.py